In [1]:
# Licence:
#     Copyright 2019 Joris Nix
    
#     Licensed under the Apache License, Version 2.0 (the "License");
#     you may not use this file except in compliance with the License.
#     You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

#     Unless required by applicable law or agreed to in writing, software
#     distributed under the License is distributed on an "AS IS" BASIS,
#     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#     See the License for the specific language governing permissions and
#     limitations under the License.

# Author: Joris Nix <joris.nix@bigdata.uni-saarland.de>

# Dynamic Programming Example in the context of Shallow and Deep Query Optimisation

In this notebook, we contrast query plans generated by Shallow Query Optimisation (SQO) to query plans generated by Deep Query Optimisation (DQO). Consider the following query:

```SQL
SELECT R.A, COUNT(*)
FROM R JOIN S ON R.ID=S.R_ID
GROUP BY R.A;
```

We assume $|R|=40,000$, $|S|=90,000$, $|R\bowtie S|=90,000$, and the number of distinct values in $R.A$ to be $20,000$. 

### Algorithms
- **Hash-based.** Traditional hash-based implementation.

- **Static Perfect Hash-based.** Uses the grouping/join key as offset into an array storing the groups, acting as a static and perfect hash function.

- **Order-based.** This implementation requires the input data to be partitioned by the grouping/join key.  In case of grouping, we compute the aggregates for each co-group . In case of a join, this represents a co-group join, i.e. we iterate over the co-groups of both inputs and compute the join result within each co-group.

- **Sort \& Order-based.** We do not require that the input data is partitioned by the grouping key. Therefore, we first sort the data (creating an order) and then we apply OG.

- **Binary Search-based.** We require to know each distinct value to initialise an array with the grouping/join keys in sorted order. This allows us to perform a binary search to lookup a group by its key.

**Note:** We only implement the cost models for each algorithm according to their algorithmic complexity as a concrete implementation of the algorithms is not needed here.


### Methodology

We consider different data properties for each input relation and result set. While SQO only considers **sortedness**, DQO considers both **sortedness** as well as **density**. This enables the use of Static Perfect Hashing (SPH). For each different combination of data properties, we compute the dynamic programming table for both SQO and DQO. Afterwards, we compute the improvement factors for the estimated plan costs of DQO over SQO. Notice that due to the foreign key join, we assume both inputs to be either dense or sparse pruning some of the possible combinations.

In [2]:
# imports
import math
import itertools

In [3]:
class Operator:
    """ 
    Operator class representing query plans.
        
    A query plan consists of nested operators.
    These operators are either unary or binary.
    """
    def __init__(self):
        pass
    
    def __str__(self):
        pass
    
    def costs(self):
        pass

class UnaryOperator(Operator):
    def __init__(self, inp):
        self._input = inp
        
    def __str__(self):
        pass
    
    def costs(self):
        pass
        
class BinaryOperator(Operator):
    def __init__(self, l_input, r_input):
        self._l_input = l_input
        self._r_input = r_input
        
    def __str__(self):
        pass
    
    def costs(self):
        pass
        
######################################## Base Relation ########################################
class Relation(Operator):
    """
    Instantiates an input relation.
        
    Attributes:
        _name (string): Name of the relation.
        _size (int): Number of tuples in the relation.
        _dense (bool): True if the relation is dense. If this attribute is None, we do SQO.
        _sorted (bool): True if the relation is sorted.
    """ 
    def __init__(self, name, size, dense, sort):
        self._name = name
        self._size = size
        self._dense = dense
        self._sorted = sort
        
    def __str__(self):
        if (self._dense is None): # dense = None means we are in SQO
            S = 'S' if self._sorted else 'US'
            return f"{self._name}_({S})"
        D = 'D' if self._dense else 'SP'
        S = 'S' if self._sorted else 'US'
        return f"{self._name}_({D},{S})"
    
    def costs(self):
        return 0

######################################## Grouping ########################################
class Grouping(UnaryOperator):
    """
    Super class for the different grouping implemenations. 
    The actual operation is not performed, only the corresponding costs are computed
    and the data properties are set accordingly..
        
    Attributes:
        _input (Operator): Input data on which the grouping operation is performed.
        _size (int): Number of groups (distinct values in self._input).
        _dense (bool): Indicates if the grouping operation produces a dense result.
        _sorted (bool): Indicates if the grouping operation produces a sorted output.
    """
    def __init__(self, inp):
        super().__init__(inp)
        self._size = 20000 # set number of groups (outputsize of grouping operation)
        
    def __str__(self):
        return f"Grouping({self._input})"
    
    def costs(self):
        pass
    
class HG(Grouping):
    """ Hash-based Grouping. """
    def __init__(self, inp):
        super().__init__(inp)
        self._sorted = False
        self._dense = self._input._dense # data density is unaffected by the grouping operation
    
    def __str__(self):
        return f"HG({self._input})"
    
    def costs(self):
        """ Cost model: HG(R) = 4 * |R| """
        cost = self._input.costs()
        return round(cost + 4 * self._input._size)
    
class SPHG(Grouping):
    """ Static Perfect Hash-based Grouping. """
    def __init__(self, inp):
        super().__init__(inp)
        self._sorted = True # SPHG always produces a sorted output
        self._dense = self._input._dense # data density is unaffected by the grouping operation
    
    def __str__(self):
        return f"SPHG({self._input})"
    
    def costs(self):
        """ Cost model: SPHG(R) = |R| """
        cost = self._input.costs()
        inp_size = self._input._size
        if (not self._input._dense): # SPHG can only be applied in a dense domain
            return float('inf')      # if sparse, infinite cost
        return round(cost + inp_size)
    
class OG(Grouping):
    """ Order-based Grouping. """
    def __init__(self, inp):
        super().__init__(inp)
        self._sorted = self._input._sorted # the ordered input does not necessarily have to be sorted
        self._dense = self._input._dense # data density is unaffected by the grouping operation
    
    def __str__(self):
        return f"OG({self._input})"
    
    def costs(self):
        """ Cost model: OG(R) = |R| """
        cost = self._input.costs()
        inp_size = self._input._size
        if (not self._input._sorted): # OG can only be applied on a orderd (sorted) input
            return float('inf')       # if unordered (unsorted), infinite cost
        return round(cost + inp_size)
    
class SOG(Grouping):
    """ Sort & Order-based Grouping. """
    def __init__(self, inp):
        super().__init__(inp)
        self._sorted = True # SOG always produces a sorted output
        self._dense = self._input._dense # data density is unaffected by the grouping operation
    
    def __str__(self):
        return f"SOG({self._input})"
    
    def costs(self):
        """ Cost model: SOG(R) = |R| * log2(|R|) + |R| """
        cost = self._input.costs()
        inp_size = self._input._size
        # the cost for sorting is only incurred, if the input is not already sorted
        return round(cost + (1 - self._input._sorted) * inp_size * math.log2(inp_size) + inp_size)
    
class BSG(Grouping):
    """ Binary Search-based Grouping. """
    def __init__(self, inp):
        super().__init__(inp)
        self._sorted = True # BSG always produces a sorted output
        self._dense = self._input._dense # data density is unaffected by the grouping operation
    
    def __str__(self):
        return f"BSG({self._input})"
    
    def costs(self):
        """ Cost model: BSG(R) = |R| * log2(#groups) """
        cost = self._input.costs()
        inp_size = self._input._size
        return round(cost + inp_size * math.log2(self._size))

######################################## Join ########################################
class Join(BinaryOperator):
    """
    Super class for the different join implemenations. 
    The actual operation is not performed, only the corresponding costs are computed
    and the data properties are set accordingly.
        
    Attributes:
        _l_input (Operator): Left input data for the join operation.
        _r_input (Operator): Right input data for the join operation.
        _size (int): Number of tuples in the join results.
        _dense (bool): Indicates if the join operation produces a dense result.
        _sorted (bool): Indicates if the join operation produces a sorted output.
    """
    def __init__(self, l_input, r_input):
        super().__init__(l_input, r_input)
        self._size = 90000 # outputsize of the join operation

    def __str__(self):
        return f"Join({self._l_input},{self._r_input})"
    
class HJ(Join):
    """ Hash-based Join. """
    def __init__(self, l_input, r_input):
        super().__init__(l_input, r_input)
        self._sorted = self._r_input._sorted # join results are pipelined; if _r_input is sorted, so is the output
        self._dense = self._l_input._dense # data density is unaffected by the join operation
        
    def __str__(self):
        return f"HJ({self._l_input},{self._r_input})"
    
    def costs(self):
        """ Cost model: HJ(R,S) = 4 * (|R| + |S|) """
        cost = self._l_input.costs() + self._r_input.costs()
        return round(cost + 4 * (self._l_input._size + self._r_input._size))

class SPHJ(Join):
    """ Static Perfect Hash-based Join. """
    def __init__(self, l_input, r_input):
        super().__init__(l_input, r_input)
        self._sorted = self._r_input._sorted # join results are pipelined; if _r_input is sorted, so is the output
        self._dense = self._l_input._dense # data density is unaffected by the join operation
        
    def __str__(self):
        return f"SPHJ({self._l_input},{self._r_input})"
    
    def costs(self):
        """ Cost model: SPHJ(R,S) = |R| + |S| """
        cost = self._l_input.costs() + self._r_input.costs()
        l_size = self._l_input._size
        r_size = self._r_input._size
        if (not self._l_input._dense): # SPHJ can only be applied in a dense domain
            return float('inf')
        return round(cost + l_size + r_size)

class OJ(Join):
    """ Order-based Join. 
        This join requires both inputs to have the same ordering.
    """
    def __init__(self, l_input, r_input):
        super().__init__(l_input, r_input)
        self._sorted = self._l_input._sorted # if either input is sorted, so is the output
        self._dense = self._l_input._dense # data density is unaffected by the join operation
        
    def __str__(self):
        return f"OJ({self._l_input},{self._r_input})"
    
    def costs(self):
        """ Cost model: OJ(R,S) = |R| + |S| """
        cost = self._l_input.costs() + self._r_input.costs()
        l_size = self._l_input._size
        r_size = self._r_input._size
        if (not self._l_input._sorted or not self._r_input._sorted): # both inputs have to be ordered/sorted
            return float('inf')
        return round(cost + (l_size + r_size))
                     
class SOJ(Join):
    """ Sort & Order-based Join. """
    def __init__(self, l_input, r_input):
        super().__init__(l_input, r_input)
        self._sorted = True # SOJ always produces a sorted output
        self._dense = self._l_input._dense # data density is unaffected by the join operation
        
    def __str__(self):
        return f"SOJ({self._l_input},{self._r_input})"
    
    def costs(self):
        """ Cost model: SOJ(R,S) = |R| * log2(|R|) + |S| * log2(|S|) + |R| + |S| """
        cost = self._l_input.costs() + self._r_input.costs()
        l_size = self._l_input._size
        r_size = self._r_input._size
        # the cost for sorting are only incurred, if the input is not already sorted
        return round(cost \
            + (1 - self._l_input._sorted) * (l_size * math.log2(l_size)) \
            + (1 - self._r_input._sorted) * (r_size * math.log2(r_size)) \
            + (l_size + r_size))
    
class BSJ(Join):
    """ Binary Search-based Join. """
    def __init__(self, l_input, r_input):
        super().__init__(l_input, r_input)
        self._sorted = self._r_input._sorted # join results are pipelined; if _r_input is sorted, so is the output
        self._dense = self._l_input._dense # data density is unaffected by the join operation
        
    def __str__(self):
        return f"BSJ({self._l_input},{self._r_input})"
    
    def costs(self):
        """ Cost model: |R| * log2(#groups) + |S| * log2(#groups)"""
        cost = self._l_input.costs() + self._r_input.costs()
        l_size = self._l_input._size
        r_size = self._r_input._size
        return round(cost + l_size * math.log2(l_size) + r_size * math.log2(l_size))

## Dynamic Programming
Define the necessary utility functions for the computation of the Dynamic Programming Table.

In [4]:
def prune_join_plans(plans):
    """ 
    This function prunes dispensable (more expensive) join plans. 
    
    The pruning is performed for a given set of input properties,
    e.g. R is dense and sorted and S is dense but unsorted.
    
    Args:
        plans (list of 'Operator'): List of plans.
    """
    kept_plans = list()
    # Group plans by their output properties.
    categorized_plans = dict() # mapping of output properties (dense, sorted) to plans
    for plan in plans:
        out_prop = (plan._dense, plan._sorted) # extract output properties from plan
        categorized_plans.setdefault(out_prop,[]).append(plan) # add to corresponding group
    # Within each group, only keep the cheapest plan
    for out_prop, plans in categorized_plans.items():
        cheapest_plan = plans[0]
        for plan in plans:
            if (plan.costs() < cheapest_plan.costs()):
                cheapest_plan = plan
        categorized_plans[out_prop] = cheapest_plan
        
    # If a sorted plan is cheaper, we can discard the unsorted plan
    sorted_output_plan = None
    unsorted_output_plan = None
    for out_prop, plan in categorized_plans.items():
        if out_prop[1]: # if join produces a sorted output
            sorted_output_plan = plan
        else:
            unsorted_output_plan = plan
            
    if unsorted_output_plan is None:
        # There are no plans that produce an unsorted output, return the sorted output plan
        kept_plans.append(sorted_output_plan)
        return kept_plans
    if (sorted_output_plan.costs() <= unsorted_output_plan.costs()):
        # The plan that produces a sorted output is cheaper, so we only keep that plan
        kept_plans.append(sorted_output_plan)
    else:
        # The plan that produces a sorted output is not cheaper, so we need to keep both plans
        kept_plans.append(sorted_output_plan)
        kept_plans.append(unsorted_output_plan)
    return kept_plans

def compute_combinations(base_rels):
    """
    Generate all possible combinations of input data properties. 
    
    Because of the foreign key join, we assume both inputs to have the same density.
    
    Args:
        base_rels (list of 'Relation'): List of all input relations with their properties to be considered by DP.
    """
    join_combinations = set()
    for comb in itertools.combinations(base_rels, 2):
        if(comb[0]._name != comb[1]._name and comb[0]._dense == comb[1]._dense):
            join_combinations.add(comb)
    return join_combinations

def display_plans(plans):
    """
    Display a plan, its cost, outputsize, density and sortedness. 
    
    Args:
        plans (list of 'Operator'): List of plans.
    """
    for p in plans:
        print(f'\t{str(p).ljust(25, " ")}\tCosts: {p.costs()}\tOutputsize: {str(p._size).ljust(5, " ")}\tDense: {p._dense}\tSorted: {p._sorted}')              

def display_dynamic_programming_table(plans):
    """
    Display the dynamic programming table.
    For each combination of input data properties, display first the join plans,
    then the grouping plans.
    
    Args:
        plans (dict of ('Relation', 'Relation') -> (joins, kept_joins, groupings)):
            Dictionary mapping an input combination to a tuple of generated join plans,
            kept_plans (after pruning), and grouping plans.
    """
    for comb, (joins, kept_joins, groupings) in plans.items():
        print(comb[0], comb[1])
        print('R⋈S')
        display_plans(joins)
        print('Γ(R⋈S)')
        display_plans(groupings)
        print('\t' + 100 * '_')    

def compute_join_plans(plans, join_algorithms):
    """
    Compute for each combination of input data properties all possible join plans
    and prune them afterwards.
    
    Args:
        plans (dict of ('Relation', 'Relation') -> (joins, kept_joins, groupings)):
            Dictionary mapping an input combination to a tuple of generated join plans,
            kept_plans (after pruning), and grouping plans.
        join_algorithms (list of 'Join'): List of join algorithms.
    """
    for comb, (joins, kept_plans, groupings) in plans.items():
        possible_plans = list()
        for alg in join_algorithms:
            joins.append(alg(comb[0], comb[1]))
        k_plans = prune_join_plans(joins)
        for kept_plan in k_plans:
              kept_plans.append(kept_plan)

def compute_grouping_plans(plans, grouping_algorithms):
    """
    Compute for each combination of input data properties all possible grouping plans
    on the kept join plans.
    
    Args:
        plans (dict of ('Relation', 'Relation') -> (joins, kept_joins, groupings)):
            Dictionary mapping an input combination to a tuple of generated join plans,
            kept_plans (after pruning), and grouping plans.
        grouping_algorithms (list of 'Grouping'): List of grouping algorithms.
    """
    for comb, (joins, kept_plans, groupings) in plans.items():
        for j in kept_plans:
            for g in grouping_algorithms:
                groupings.append(g(j))

### Shallow Query Optimisation

In [5]:
# Initialise base relations with their according properties
base_relations = [Relation('R', 40000, None, True), Relation('R', 40000, None, False),
                  Relation('S', 90000, None, True), Relation('S', 90000, None, False)]

combinations = compute_combinations(base_relations)
plans = dict()  # mapping of input combination to tuple containing a list of join plans,
                # a list of kept plans, and a list of grouping plans
for comb in combinations:
    plans[comb] = ([], [], [])
    
# Compute join plans
join_algorithms = [HJ, OJ, SOJ, BSJ]
compute_join_plans(plans, join_algorithms)

# Compute grouping plans on top of the join plans
grouping_algorithms = [HG, OG, SOG, BSG]
compute_grouping_plans(plans, grouping_algorithms)

# Display dynamic programming table
display_dynamic_programming_table(plans)

R_(US) S_(S)
R⋈S
	HJ(R_(US),S_(S))         	Costs: 520000	Outputsize: 90000	Dense: None	Sorted: True
	OJ(R_(US),S_(S))         	Costs: inf	Outputsize: 90000	Dense: None	Sorted: False
	SOJ(R_(US),S_(S))        	Costs: 741508	Outputsize: 90000	Dense: None	Sorted: True
	BSJ(R_(US),S_(S))        	Costs: 1987403	Outputsize: 90000	Dense: None	Sorted: True
Γ(R⋈S)
	HG(HJ(R_(US),S_(S)))     	Costs: 880000	Outputsize: 20000	Dense: None	Sorted: False
	OG(HJ(R_(US),S_(S)))     	Costs: 610000	Outputsize: 20000	Dense: None	Sorted: True
	SOG(HJ(R_(US),S_(S)))    	Costs: 610000	Outputsize: 20000	Dense: None	Sorted: True
	BSG(HJ(R_(US),S_(S)))    	Costs: 1805894	Outputsize: 20000	Dense: None	Sorted: True
	____________________________________________________________________________________________________
R_(S) S_(US)
R⋈S
	HJ(R_(S),S_(US))         	Costs: 520000	Outputsize: 90000	Dense: None	Sorted: False
	OJ(R_(S),S_(US))         	Costs: inf	Outputsize: 90000	Dense: None	Sorted: True
	SOJ(R_(S),S_(US))

### Deep Query Optimisation

In [6]:
# Initialise base relations with their according properties
base_relations = [Relation('R', 40000, True, True), Relation('R', 40000, True, False),
                  Relation('R', 40000, False, True), Relation('R', 40000, False, False),
                  Relation('S', 90000, True, True), Relation('S', 90000, True, False),
                  Relation('S', 90000, False, True), Relation('S', 90000, False, False)]


combinations = compute_combinations(base_relations)
plans = dict()  # mapping of input combination to tuple containing a list of join plans,
                # a list of kept plans, and a list of grouping plans
for comb in combinations:
    plans[comb] = ([], [], [])

# Compute join plans
join_algorithms = [HJ, SPHJ, OJ, SOJ, BSJ]
compute_join_plans(plans, join_algorithms)

# Compute grouping plans on top of the join plans
grouping_algorithms = [HG, SPHG, OG, SOG, BSG]
compute_grouping_plans(plans, grouping_algorithms)

# Display dynamic programming table
display_dynamic_programming_table(plans)

R_(SP,S) S_(SP,US)
R⋈S
	HJ(R_(SP,S),S_(SP,US))   	Costs: 520000	Outputsize: 90000	Dense: False	Sorted: False
	SPHJ(R_(SP,S),S_(SP,US)) 	Costs: inf	Outputsize: 90000	Dense: False	Sorted: False
	OJ(R_(SP,S),S_(SP,US))   	Costs: inf	Outputsize: 90000	Dense: False	Sorted: True
	SOJ(R_(SP,S),S_(SP,US))  	Costs: 1611187	Outputsize: 90000	Dense: False	Sorted: True
	BSJ(R_(SP,S),S_(SP,US))  	Costs: 1987403	Outputsize: 90000	Dense: False	Sorted: False
Γ(R⋈S)
	HG(SOJ(R_(SP,S),S_(SP,US)))	Costs: 1971187	Outputsize: 20000	Dense: False	Sorted: False
	SPHG(SOJ(R_(SP,S),S_(SP,US)))	Costs: inf	Outputsize: 20000	Dense: False	Sorted: True
	OG(SOJ(R_(SP,S),S_(SP,US)))	Costs: 1701187	Outputsize: 20000	Dense: False	Sorted: True
	SOG(SOJ(R_(SP,S),S_(SP,US)))	Costs: 1701187	Outputsize: 20000	Dense: False	Sorted: True
	BSG(SOJ(R_(SP,S),S_(SP,US)))	Costs: 2897081	Outputsize: 20000	Dense: False	Sorted: True
	HG(HJ(R_(SP,S),S_(SP,US)))	Costs: 880000	Outputsize: 20000	Dense: False	Sorted: False
	SPHG(HJ(R_(SP,S),

### Summary

In this notebook, we computed the dynamic programming table for a concrete example in the context of SQO and DQO for different combinations of input data properties (sortedness and density).

The following table shows the improvement factor for the estimated plan costs of DQO over SQO for the different settings.


| <i></i>        | <i></i>        | sparse | dense  |
| ---------------| ---------------| -------| -------|
| $R_{sorted}$   | $S_{sorted}$   | $1x$   | $1x$   |
| $R_{sorted}$   | $S_{unsorted}$ | $1x$   | $4x$   |
| $R_{unsorted}$ | $S_{sorted}$   | $1x$   | $2.8x$ |
| $R_{unsorted}$ | $S_{unsorted}$ | $1x$   | $4x$   |